In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

# Selenium WebDriver 
service = Service(r"C:\Users\theoi\Downloads\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service)




In [ ]:
# FPL Review planner page
driver.get("https://fplreview.com/free-planner/#teamProjections")

# wait for the page to load and elements to appear
wait = WebDriverWait(driver, 20)



In [ ]:

team_id_input = wait.until(EC.element_to_be_clickable((By.ID, "liveTeamID")))
team_id_input.clear()
team_id_input.send_keys("1")


load_page_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "redbutton")))
load_page_button.click()


load_group_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "myGroup")))
from selenium.webdriver.support.ui import Select
select = Select(load_group_dropdown)
select.select_by_visible_text("All Players")

xmins_option = wait.until(EC.element_to_be_clickable((By.ID, "XMINopt")))
xmins_option.click()


In [ ]:
table = wait.until(EC.presence_of_element_located((By.ID, "scout_table")))
rows = table.find_elements(By.CSS_SELECTOR, "tr.playerRow")
data = []
 
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = []
    for cell in cells:
        # if the cell contains a player name/details, extract those specifically
        player_name = cell.find_elements(By.CLASS_NAME, "playerName")
        player_details = cell.find_elements(By.CLASS_NAME, "playerDetails")
        if player_name and player_details:
            row_data.append(player_name[0].text)
            row_data.append(player_details[0].text)
        else:
            row_data.append(cell.text)
    data.append(row_data)


In [ ]:
# define columns to match table structure
columns = [
    "index", "name", "details", "xmins", "xmins_gw1", "xmins_gw2", "xmins_gw3", "xmins_gw4", "xmins_gw5","total_xmins","/£M","elite%","empty"
]
df = pd.DataFrame(data, columns=columns[:len(data[0])]).drop(columns = ["empty"])
print(df)

    index                 name  details xmins xmins_gw1 xmins_gw2 xmins_gw3  \
0                     Petrović   GK 4.5    94        94        90        94   
1                        Roefs   GK 4.5    94        94        91        90   
2                      Sánchez   GK 5.0    94        94        92        90   
3                      José Sá   GK 4.5    93        93        94        93   
4                   Verbruggen   GK 4.5    93        93        93        92   
..    ...                  ...      ...   ...       ...       ...       ...   
691        Josh Wilson-Esbrand  DF 99.9     0         0         0         0   
692                Issa Kaboré  DF 99.9     0         0         0         0   
693               Ben Chilwell  DF 99.9     0         0         0         0   
694           Hindolo Mustapha  MD 99.9     0         0         0         0   
695               Callum Doyle  DF 99.9     0         0         0         0   

    xmins_gw4 xmins_gw5 total_xmins   /£M elite%  


In [ ]:
# split details into position and price columns
df[['position', 'price']] = df['details'].str.extract(r'(\w+)\s+([\d.]+)')
df = df.drop(columns=['details'])
print(df[['name', 'position', 'price']].head())

         name position price
0    Petrović       GK   4.5
1       Roefs       GK   4.5
2     Sánchez       GK   5.0
3     José Sá       GK   4.5
4  Verbruggen       GK   4.5


In [32]:
print(df)

    index                 name xmins xmins_gw1 xmins_gw2 xmins_gw3 xmins_gw4  \
0                     Petrović    94        94        90        94        90   
1                        Roefs    94        94        91        90        91   
2                      Sánchez    94        94        92        90        87   
3                      José Sá    93        93        94        93        92   
4                   Verbruggen    93        93        93        92        92   
..    ...                  ...   ...       ...       ...       ...       ...   
691        Josh Wilson-Esbrand     0         0         0         0         0   
692                Issa Kaboré     0         0         0         0         0   
693               Ben Chilwell     0         0         0         0         0   
694           Hindolo Mustapha     0         0         0         0         0   
695               Callum Doyle     0         0         0         0         0   

    xmins_gw5 total_xmins   /£M elite% 